# Acquire Sentinel-2 MSI Data for California
## Capstone Fall 2020
### TP Goter

This notebook is used for gathering data from California from the Sentinel-2 satellites. Specifically, we are looking to acquire the surface reflectance data (atmosphere corrected - level 2a) as that is what we did our baseline model testing and evaluation with using the Big Earth Net data. We will gather data from a variety of geographic areas but most of our focus will be on the agricultural regions of California (e.g., Central Valley).

In [6]:
import time
from math import sin, cos, sqrt, atan2, radians
import pandas as pd
import ee
from shapely.geometry import box
import folium
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import find_peaks


In [4]:
def authenticate():
    # Trigger the authentication flow.
    ee.Authenticate()


In [47]:
class MSICalifornia():

    def __init__(self, center_lat=43.771114, center_lon=-116.736866, edge_len=0.005, year=2019):
        '''
        Parameters:
            center_lat: latitude for the location coordinate
            center_lon: longitude for the location coordinate
            edge_len: edge length in degrees for the rectangle given the location coordinates
            year: year the satellite data should pull images for
            '''

        # Initialize the library.
        ee.Initialize()

        # Error handle parameter issues
        if center_lat >= -90 and center_lat <= 90:
            self.center_lat = center_lat
        else:
            raise ValueError('Please enter float value for latitude between -90 and 90')
            exit()

        if center_lon >= -180 and center_lon <= 180:
            self.center_lon = center_lon
        else:
            raise ValueError('Please enter float value for longitude between -180 and 180')
            exit()

        if (type(edge_len) == float and (edge_len <= 0.5 and edge_len >= 0.005)):
            self.edge_len = edge_len
        else:
            raise ValueError('Please enter float value for edge length between 0.5 and 0.005')
            exit()

        # (range is 2017 to year prior)
        if ((type(year) == int) and (year >= 2017 and year <= int(time.strftime("%Y")) - 1)):
            self.year = year
        else:
            raise ValueError(
                'Please enter an integer value for year > 2017 and less than the current year')
            exit()

        # initialize remaining variables
        self.label = []
        self.comment = dict()
        self.image = ee.Image()
        self.simple_image = ee.Image()
        self.base_asset_directory = None

        # Create the bounding box using GEE API
        self.aoi_ee = self.__create_bounding_box_ee()
        # Estimate the area of interest
        self.dist_lon = self.__calc_distance(
            self.center_lon - self.edge_len / 2, self.center_lat, self.center_lon + self.edge_len / 2, self.center_lat)
        self.dist_lat = self.__calc_distance(
            self.center_lon, self.center_lat - self.edge_len / 2, self.center_lon, self.center_lat + self.edge_len / 2)
        print('The selected area is approximately {:.2f} km by {:.2f} km'.format(
            self.dist_lon, self.dist_lat))

        self.model_projection = "EPSG:3857"

    def __create_bounding_box_ee(self):
        '''Creates a rectangle for pulling image information using center coordinates and edge_len'''
        return ee.Geometry.Rectangle([self.center_lon - self.edge_len / 2, self.center_lat - self.edge_len / 2, self.center_lon + self.edge_len / 2, self.center_lat + self.edge_len / 2])

    def __create_bounding_box_shapely(self):
        '''Returns a box for coordinates to plug in as an image add-on layer'''
        return box(self.center_lon - self.edge_len / 2, self.center_lat - self.edge_len / 2, self.center_lon + self.edge_len / 2, self.center_lat + self.edge_len / 2)

    @staticmethod
    def __calc_distance(lon1, lat1, lon2, lat2):
        '''Calculates the distance between 2 coordinates'''
        # Reference: https://stackoverflow.com/questions/19412462/getting-distance-between-two-points-based-on-latitude-longitude
        # approximate radius of earth in km
        R = 6373.0
        lon1 = radians(lon1)
        lat1 = radians(lat1)
        lon2 = radians(lon2)
        lat2 = radians(lat2)
        dlon = lon2 - lon1
        dlat = lat2 - lat1
        a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
        c = 2 * atan2(sqrt(a), sqrt(1 - a))
        distance = R * c
        return distance

    def pull_Sentinel2_data(self):
       
        
        # 10 of 13 Spectral Bands are retained. 10th band has no surface reflectance per
        # http://bigearth.net/static/documents/BigEarthNet_IGARSS_2019.pdf
        
        # Also the baseline model only used the 10 and 20m bands (remove band 1 and 9)
        band_names = ['B2', 'B3', 'B4', 'B5',
                      'B6', 'B7', 'B8', 'B8A', 'B9',
                      'B11', 'B12']
        
        random_month = np.random.randint(1,13)

        start_date = f'{self.year}-{random_month}-01'
        if random_month != 12:
            end_date = f'{self.year}-{random_month+1}-01'
        else:
            end_date = f'{self.year +1 }-1-01'

        self.Sentinel_MSI = (ee.ImageCollection('COPERNICUS/S2_SR')
                             .filterDate(start_date, end_date)
                             .filterBounds(self.aoi_ee)
                             .select(band_names)
                             .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 1))
                             .median().clip(self.aoi_ee))
        return random_month


    def plot_map(self):
        '''Plot folium map using GEE api - the map includes are of interest box and associated ndvi readings'''

        def add_ee_layer(self, ee_object, vis_params, show, name):
            '''Checks if image object classifies as ImageCollection, FeatureCollection, Geometry or single Image
            and adds to folium map accordingly'''
            try:
                if isinstance(ee_object, ee.image.Image):
                    map_id_dict = ee.Image(ee_object).getMapId(vis_params)
                    folium.raster_layers.TileLayer(
                        tiles=map_id_dict['tile_fetcher'].url_format,
                        attr='Google Earth Engine',
                        name=name,
                        overlay=True,
                        control=True,
                        show=show
                    ).add_to(self)
                elif isinstance(ee_object, ee.imagecollection.ImageCollection):
                    ee_object_new = ee_object.median()
                    map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
                    folium.raster_layers.TileLayer(
                        tiles=map_id_dict['tile_fetcher'].url_format,
                        attr='Google Earth Engine',
                        name=name,
                        overlay=True,
                        control=True,
                        show=show
                    ).add_to(self)
                elif isinstance(ee_object, ee.geometry.Geometry):
                    folium.GeoJson(
                        data=ee_object.getInfo(),
                        name=name,
                        overlay=True,
                        control=True
                    ).add_to(self)
                elif isinstance(ee_object, ee.featurecollection.FeatureCollection):
                    ee_object_new = ee.Image().paint(ee_object, 0, 2)
                    map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
                    folium.raster_layers.TileLayer(
                        tiles=map_id_dict['tile_fetcher'].url_format,
                        attr='Google Earth Engine',
                        name=name,
                        overlay=True,
                        control=True,
                        show=show
                    ).add_to(self)

            except:
                print("Could not display {}".format(name))

        # Add EE drawing method to folium.
        folium.Map.add_ee_layer = add_ee_layer

        myMap = folium.Map(location=[self.center_lat, self.center_lon], zoom_start=11)
        aoi_shapely = self.__create_bounding_box_shapely()
        folium.GeoJson(aoi_shapely, name="Area of Interest").add_to(myMap)

        
        # Add Sentinel-2 RGB quarterly layers
        start = time.time()
        visParams = {'max': 4000}
        # Add MSI layer for July
        myMap.add_ee_layer(self.Sentinel_MSI.select(['B2','B3','B4']), visParams, show=False, name="Sentinel2A")
        end = time.time()
        print("ADDED S2 RGB LAYERS \t\t--> " + str(round((end - start) / 60, 2)) + " min")

        return myMap


    def write_image_google_drive(self, filename):
        '''Writes predicted image out as an image to Google Drive as a TIF file'''
        bands = ['B2', 'B3', 'B4', 'B5',
                'B6', 'B7', 'B8', 'B8A',
                'B11', 'B12']
        tasks = []
        task = ee.batch.Export.image.toDrive(
            crs=self.model_projection,
            region=self.aoi_ee,
            image=self.Sentinel_MSI.select(bands),
            description=filename)
        print(f"Writing To Google Drive filename = {filename}.tif")
        task.start()

        

In [9]:
authenticate()

Enter verification code:  4/5AHigCGsN7yW3nKmAXiOLE3S05Gvw1mpt8WC4m7MnLT7iAprgw48ZxA



Successfully saved authorization token.


## Degree to distance calculation
- One degree of latitude equals approximately 364,080 feet (69 miles), one minute equals 6,068 feet (1.15 miles), and one-second equals 101 feet. 
- One-degree of longitude equals 288,200 feet (54.6 miles), one minute equals 4,800 feet (0.91 mile), and one second equals 80 feet.
- 1.60934 km per mile
- 9748 square kilometers per squared degree

In [48]:
# Latitude and Longitude of center point
edge_len = 0.25

# Grab Central Valley region of California

# Fresno to Bakersfield
#lat_range = np.arange(35.125,37.125,edge_len)
#lon_range = np.arange(-119.875,-118.875,edge_len)

# Sacramento to Merced
#lat_range = np.arange(37.125,38.625,edge_len)
lat_range = np.arange(37.125,37.375,edge_len)
lon_range = np.arange(-121.125,-120.875,edge_len)

year = 2019

# Iterate over range of lats and longs
for lat in lat_range:
    for lon in lon_range:
        # Instantiate the model
        print(f'Evaluating irrigation at {lat}, {lon}')
        
        # Instantiate the model
        model = MSICalifornia(
            center_lat=lat, 
            center_lon=lon, 
            edge_len=edge_len, 
            year=year)
        
        month = model.pull_Sentinel2_data()
                   
        base_filename = f'S2SR_{month}_{year}_{lat}_{lon}'    

        model.write_image_google_drive(base_filename)
        

Evaluating irrigation at 37.125, -121.125
The selected area is approximately 22.17 km by 27.81 km
Writing To Google Drive filename = S2SR_3_2019_37.125_-121.125.tif


In [49]:
%%time
model.plot_map()

ADDED S2 RGB LAYERS 		--> 0.0 min
CPU times: user 7.3 ms, sys: 7.84 ms, total: 15.1 ms
Wall time: 238 ms


In [50]:
rgb_img = model.Sentinel_MSI.select(['B2','B3','B4'])

In [51]:
rgb_img.getInfo()

{'type': 'Image',
 'bands': [{'id': 'B2',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    'min': 0,
    'max': 65535},
   'dimensions': [2, 1],
   'origin': [-122, 37],
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'B3',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    'min': 0,
    'max': 65535},
   'dimensions': [2, 1],
   'origin': [-122, 37],
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'B4',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    'min': 0,
    'max': 65535},
   'dimensions': [2, 1],
   'origin': [-122, 37],
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]}],
 'properties': {'system:footprint': {'type': 'Polygon',
   'coordinates': [[[-121.25, 37],
     [-121, 37],
     [-121, 37.25],
     [-121.25, 37.25],
     [-121.25, 37]]]}}}

In [33]:
model.Sentinel_MSI.bandTypes().getInfo()

{'B11': {'type': 'PixelType', 'precision': 'double', 'min': 0, 'max': 65535},
 'B12': {'type': 'PixelType', 'precision': 'double', 'min': 0, 'max': 65535},
 'B2': {'type': 'PixelType', 'precision': 'double', 'min': 0, 'max': 65535},
 'B3': {'type': 'PixelType', 'precision': 'double', 'min': 0, 'max': 65535},
 'B4': {'type': 'PixelType', 'precision': 'double', 'min': 0, 'max': 65535},
 'B5': {'type': 'PixelType', 'precision': 'double', 'min': 0, 'max': 65535},
 'B6': {'type': 'PixelType', 'precision': 'double', 'min': 0, 'max': 65535},
 'B7': {'type': 'PixelType', 'precision': 'double', 'min': 0, 'max': 65535},
 'B8': {'type': 'PixelType', 'precision': 'double', 'min': 0, 'max': 65535},
 'B8A': {'type': 'PixelType', 'precision': 'double', 'min': 0, 'max': 65535},
 'B9': {'type': 'PixelType', 'precision': 'double', 'min': 0, 'max': 65535}}